In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.optimize

from datetime import datetime
import pytz

import gm2
import trfp

Welcome to JupyROOT 6.12/04


In [2]:
station_phi = trfp.STATION_BARCODE_PHI
station_edges = trfp.STATION_BARCODE_EDGES

corrected_df_1 = trfp.remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_moment_df'))
tr_baseline_1, fp_baseline_1, baseline_time_1, summed_azimuth_1, _ = trfp.trolley_run_station_average(corrected_df_1)

corrected_df_2 = trfp.remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_2.h5', key='run_3997_moment_df'))
tr_baseline_2, fp_baseline_2, baseline_time_2, summed_azimuth_2, _ = trfp.trolley_run_station_average(corrected_df_2)

Finished removing trolley images from 4382 events.                        
Finished removing trolley images from 4359 events.                         


In [24]:
# print np.round(tr_baseline_1[0,0:5], 2)
# print np.round(tr_baseline_2[0,0:5], 2)
# print np.round(fp_baseline_1[0,0:5], 2)
# print np.round(fp_baseline_2[0,0:5], 2)

# print np.nanmean(tr_baseline_1[0,0])
# print np.nanmean(tr_baseline_2[0,0])

# print np.nanmean(fp_baseline_1[0,0])
# print np.nanmean(fp_baseline_2[0,0])

print baseline_time_1[0]
print baseline_time_2[0]

1524378742.0
1524645720.5


In [14]:
# load all fixed probe runs moment_df into one big moment_df (runs 3959--3994)
print 'Appending fixed probe runs.'
fp_moment_df = pd.read_hdf('60hr_fixed_probe_runs.h5', key='run_3959_moment_df')
pts = fp_moment_df.shape[0]
for run in np.arange(3959, 3995):
    temp_df = pd.read_hdf('60hr_fixed_probe_runs.h5', key='run_'+str(run)+'_moment_df')
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    fp_moment_df = fp_moment_df.append(temp_df)

# load all the trolley runs corrected_df into on big file (to show window of 60hr set) ()
print '\nAppending trolley runs.'
tr_corrected_df = trfp.remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3955_moment_df'))
for run in np.arange(3956, 3959):
    temp_df = trfp.remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_'+str(run)+'_moment_df'))
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)
for run in np.arange(3995, 3999):
    temp_df = trfp.remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_2.h5', key='run_'+str(run)+'_moment_df'))
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)

print '\nDone appending runs.'

### Apply baseline corrections, generate virtual trolley measurements

fp_moment_baseline = fp_moment_df.copy()
fp_moment_tr_run_baseline = tr_corrected_df.copy()

# apply baseline corrections to each fp stm
print "\nSubtracting fixed probe baselines."
for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = fp_baseline_1[st, m]
            c2 = fp_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(fp_moment_baseline.index.values)
        fp_moment_baseline[stm] = fp_moment_baseline[stm] - correction
        correction = backwards_correction(fp_moment_tr_run_baseline.index.values)
        fp_moment_tr_run_baseline[stm] = fp_moment_tr_run_baseline[stm] - correction
        
        print '\rstm: ' + stm + '.',
        
# replace columns in vtr with Jacobian-fixed columns from fp baseline correction

print "\n\nApplying Jacobian."
vtr_df = fp_moment_df.copy()
vtr_tr_run_df = tr_corrected_df.copy()

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    if num_probes == 4:
        num_moments = 4
        if st == 41:
            J = trfp.J_4_PROBE_ST41
        elif st == 37 | st == 39:
            J = trfp.J_4_PROBE_ST37_ST39
        else:
            J = trfp.J_4_PROBE
    else:
        num_moments = 5
        if st < 7:
            J = trfp.J_6_PROBE_OFFSET
        else:
            J = trfp.J_6_PROBE
    # run over each vtr moment:
    for m in np.arange(num_moments):
        vtr_stm = 'st'+str(st)+',m'+str(m+1)
        fp_stm = ['st'+str(st)+',m'+str(fp_m+1) for fp_m in np.arange(num_moments)]
        vtr_df[vtr_stm] = fp_moment_baseline[fp_stm].dot(J[m])
        vtr_tr_run_df[vtr_stm] = fp_moment_tr_run_baseline[fp_stm].dot(J[m])
        
        print '\rstm: ' + stm + '.',
        
# Add trolley baseline correction (with backwards correction)
print "\n\nAdding trolley baselines."

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = tr_baseline_1[st, m]
            c2 = tr_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(vtr_df.index.values)
        vtr_df[stm] = vtr_df[stm] + correction
        correction = backwards_correction(vtr_tr_run_df.index.values)
        vtr_tr_run_df[stm] = vtr_tr_run_df[stm] + correction
        
        print '\rstm: ' + stm + '.',

print '\n\nDone.'

Appending fixed probe runs.
Appending run 3994.                                 
Appending trolley runs.
Finished removing trolley images from 893 events.        
Finished removing trolley images from 4382 events.                         
Finished removing trolley images from 2977 events.             
Finished removing trolley images from 539 events.      
Finished removing trolley images from 365 events.    
Finished removing trolley images from 921 events.        
Finished removing trolley images from 4359 events.                        
Finished removing trolley images from 2853 events.             
Appending run 3998. 
Done appending runs.

Subtracting fixed probe baselines.
stm: st71,m4.    

Applying Jacobian.
stm: st71,m4.          

Adding trolley baselines.
stm: st71,m4.    

Done.


In [22]:
vtr_df.iloc[0]

st0,m1     51446.581383
st0,m2        96.623767
st0,m3       -90.042325
st0,m4       -41.224204
st0,m5        16.991188
st0,m6      -844.612420
st1,m1     50594.128979
st1,m2        72.782043
st1,m3        60.826868
st1,m4       376.996598
st1,m5         3.022472
st1,m6      3550.464672
st2,m1     50304.196589
st2,m2       -48.301516
st2,m3        37.413780
st2,m4       860.732452
st2,m5       -50.390084
st2,m6      -722.222822
st3,m1     51501.489273
st3,m2      -229.955724
st3,m3      -233.702878
st3,m4       553.071227
st3,m5      -416.085266
st3,m6      9079.912465
st4,m1     52405.483691
st4,m2     -1692.794166
st4,m3      -704.678120
st4,m4       -49.140896
st4,m5     -1616.687430
st4,m6      -273.212401
               ...     
st67,m1    51667.291808
st67,m2       97.580291
st67,m3       75.190201
st67,m4       13.298634
st67,m5             NaN
st67,m6             NaN
st68,m1    51361.874087
st68,m2       70.939554
st68,m3      -36.679329
st68,m4     -303.219618
st68,m5      -77

In [ ]:
# print fp_baseline_1[:,0]

tr3956 = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_moment_df')
tr3956_corrected = trfp.remove_trolley_effect(tr3956)

In [ ]:
drift_means = np.zeros(72)
# for i in range(6,54)+range(55,72):
for i in range(72):
#     print np.round((tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])/61.79,4)
    drift_means[i] = (tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])
    
drift_mean = np.mean(drift_means)
# plt.plot(drift_means/61.79, '.')
print drift_mean/61.79
print '\n'

drift_means = np.zeros(72)
# for i in range(6,54)+range(55,72):
for i in [0, 2, 4] + range(6, 54) + range(55,72):
#     print np.round((tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])/61.79,4)
    drift_means[i] = (tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])
    
drift_mean = np.sum(drift_means)/68
# plt.plot(drift_means/61.79, '.')
print drift_mean/61.79
print '\n'

drift_means = np.zeros(72)
# for i in range(6,54)+range(55,72):
for i in range(72):
#     print np.round((tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])/61.79,4)
    drift_means[i] = (tr3956.iloc[-1]['st'+str(i)+',m1'] - fp_baseline_1[i,0])

drift_means[1] = (drift_means[0] + drift_means[2])/2
drift_means[3] = (drift_means[2] + drift_means[4])/2
drift_means[5] = (drift_means[4] + drift_means[6])/2
drift_means[54] = (drift_means[53] + drift_means[55])/2

drift_mean = np.mean(drift_means)
# plt.plot(drift_means/61.79, '.')
print drift_mean/61.79
print '\n'

# # average trolley

# weight = summed_azimuth_1
# total_weight = np.sum(summed_azimuth_1)

# avg_trolley = np.dot(tr_baseline_1[:,0], summed_azimuth_1)/np.sum(summed_azimuth_1)
# print avg_trolley/61.79
# print '\n'

# trolley_drift = drift_mean

# corrected_trolley = avg_trolley + trolley_drift
# print corrected_trolley/61.79

# print drift_means[6]/61.79

# fig,ax = plt.subplots(1,1)
# plt.sca(ax)
# plt.plot(tr3956_corrected.index.values, tr3956_corrected['st6,m1']/61.79, '.')
# plt.plot(baseline_time_1[6], fp_baseline_1[6,0]/61.79, '.', color='red', markersize=12)

In [ ]:
tr_run_1 = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_moment_df')
tr_run_2 = pd.read_hdf('60hr_trolley_runs_2.h5', key='run_3997_moment_df')

In [ ]:
tr_moment = 'st0,m1'

fig, ax = plt.subplots(2,1)
plt.sca(ax[0])
plt.plot(tr_run_1['tr_phi'], tr_run_1[tr_moment]/61.79, '.', markersize=1, color='navy')
plt.sca(ax[1])
plt.plot(tr_run_2['tr_phi'], tr_run_2[tr_moment]/61.79, '.', markersize=1, color='navy')

# ax[0].set_ylim(-10,10)

fig.set_size_inches(12,8)
fig.tight_layout()

In [ ]:
tr_run_1_azi = tr_run_1.copy()
tr_run_1_azi = tr_run_1_azi.set_index('tr_phi')[['tr,m'+str(N+1) for N in range(17)]].round(3).sort_index()

tr_run_1_azi.to_csv('tr_run_3956.dat', sep=' ', index_label='azimuth', float_format='%0.4f')

tr_run_2_azi = tr_run_2.copy()
tr_run_2_azi = tr_run_2_azi.set_index('tr_phi')[['tr,m'+str(N+1) for N in range(17)]].round(3).sort_index()

tr_run_2_azi.to_csv('tr_run_3997.dat', sep=' ', index_label='azimuth', float_format='%0.4f')

In [ ]:
# The un-drift corrected trolley average

for x in np.round(np.matmul(np.transpose(tr_baseline_1),summed_azimuth_1)/360, 4):
    print x

print '\n'
    
for x in np.round(np.matmul(np.transpose(tr_baseline_2),summed_azimuth_2)/360, 4):
    print x


In [ ]:
plt.plot(vtr_df.index.values, vtr_df['st0,m1'], '.')

In [3]:
# bin into the agreed upon bins

bins = np.arange(1524384055, 1524641055, 1000)-500
bin_centers = np.arange(1524384055, 1524640055, 1000)

# bins = np.arange(1524383560, 1524640850, 10)-5
# bin_centers = np.arange(1524383560, 1524640840, 10)

vtr_time_bin_df = vtr_df.groupby(pd.cut(vtr_df.index,bins)).mean()
vtr_time_bin_df.index = bin_centers
vtr_time_bin_df.head()


# print vtr_time_bin_df.iloc[[0,-1]]
# print vtr_time_bin_df.shape
# print bin_centers.shape

,"st0,m1","st0,m2","st0,m3","st0,m4","st0,m5","st0,m6","st1,m1","st1,m2","st1,m3","st1,m4",...,"st70,m3","st70,m4","st70,m5","st70,m6","st71,m1","st71,m2","st71,m3","st71,m4","st71,m5","st71,m6"
1524384055,51448.696253,97.645631,-90.040709,-41.196627,17.921009,-843.719404,50363.755460,46.531512,46.727571,356.299032,...,-26.100485,-38.998738,-77.504125,551.273108,51421.335395,55.026796,-36.842357,53.892031,NaN,NaN
1524385055,51448.185635,97.949220,-89.985768,-41.177959,17.722281,-843.834844,50377.655358,59.389673,49.738658,357.009271,...,-26.066889,-38.978478,-77.477438,551.403500,51420.600063,55.239487,-36.845882,53.921798,NaN,NaN
1524386055,51445.919245,98.150300,-89.935324,-41.187477,17.670727,-844.133947,50384.878346,62.271261,50.032426,356.133745,...,-26.048864,-39.008041,-77.835053,551.146589,51417.029274,55.501525,-36.815045,53.875312,NaN,NaN
1524387055,51445.246408,98.339610,-89.914114,-41.174953,17.627043,-844.080069,50361.472989,52.077679,47.882137,356.058258,...,-26.109656,-38.979994,-77.493955,551.224598,51413.182155,55.784961,-36.889573,53.920346,NaN,NaN
1524388055,51444.038464,98.761892,-89.847252,-41.173662,17.640946,-844.188643,50362.244859,50.599378,46.493092,355.363219,...,-26.132680,-38.932495,-77.640462,551.436855,51408.848065,56.449467,-36.881342,53.907322,NaN,NaN


In [4]:
test_df = vtr_time_bin_df.copy()

azi_avg_df = pd.DataFrame(np.zeros((test_df.shape[0],6)),
                         index = test_df.index,
                         columns = ['m' + str(m) for m in np.arange(6)+1])

weight = summed_azimuth_1+summed_azimuth_2
total_weight = np.sum(summed_azimuth_1 + summed_azimuth_2)

for m in np.arange(6):
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df['m'+str(m+1)] = test_df[stm_list].multiply(weight).sum(axis=1)/total_weight

In [ ]:
test_df_2 = vtr_time_bin_df.copy()

azi_avg_df_2 = pd.DataFrame(np.zeros((test_df_2.shape[0],6)),
                            index = test_df.index,
                            columns = ['m' + str(m) for m in np.arange(6)+1])

weight = summed_azimuth_1+summed_azimuth_2
weight[1] = 0
weight[3] = 0
weight[5] = 0
weight[54] = 0
total_weight = np.sum(weight)

for m in np.arange(6):
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df_2['m'+str(m+1)] = test_df_2[stm_list].multiply(weight).sum(axis=1)/total_weight

In [ ]:
print test_df_2['st1,m1'].iloc[0]
print test_df_2['st3,m1'].iloc[0]
print test_df_2['st5,m1'].iloc[0]
print test_df_2['st54,m1'].iloc[0]

print (test_df_2['st1,m1'].iloc[0] + test_df_2['st3,m1'].iloc[0] + test_df_2['st5,m1'].iloc[0] + test_df_2['st54,m1'].iloc[0])/4/61.79

print azi_avg_df.iloc[0]/61.79
print azi_avg_df_2.iloc[0]/61.79

In [ ]:
review_df = azi_avg_df[['m1', 'm2', 'm3', 'm5']].copy()

review_df['m1_unc'] = 0.116
review_df['m2_unc'] = 0.060
review_df['m3_unc'] = 0.062
review_df['m5_unc'] = 0.100

review_df['m1'] = review_df['m1']/61.79
review_df['m2'] = review_df['m2']/61.79
review_df['m3'] = review_df['m3']/61.79
review_df['m5'] = review_df['m5']/61.79

review_df = review_df.reindex(sorted(review_df.columns), axis=1)

print review_df.head()

In [ ]:
review_df.to_csv('purcell_60hr_unblinded_1000sec_bins_new_calibrations.dat', sep=' ', index_label='time', float_format='%0.4f')

In [ ]:
print trfp.PLUNGING_PROBE_CALIBRATIONS

In [ ]:
plt.plot(review_df.index.values, review_df['m2'], '.')
fig = plt.gcf()
fig.set_size_inches(12,4)
fig.tight_layout()

In [ ]:
print vtr_df.shape
print vtr_tr_run_df.shape

vtr_total_df = vtr_tr_run_df.copy()
vtr_total_df = vtr_total_df.drop(axis=1, labels=['tr_phi']+['tr,m'+str(n+1) for n in range(17)])

vtr_total_df = pd.concat([vtr_total_df, vtr_df])
vtr_total_df = vtr_total_df.sort_index()
vtr_total_df = vtr_total_df.loc[~vtr_total_df.index.duplicated()]  # drop any accidentally duplicated rows

print vtr_total_df.shape

In [ ]:
for i in vtr_total_df.index.values[0:5]: print i
for i in vtr_total_df.index.values[-5:]: print i

In [ ]:
# bin into the agreed upon bins

# bins = np.arange(1524375055, 1524649155, 1000)-500
# bin_centers = np.arange(1524375055, 1524649055, 1000)

bins = np.arange(1524375055, 1524649155, 50)-25
bin_centers = np.arange(1524375055, 1524649155-50, 50)

vtr_time_bin_df = vtr_total_df.groupby(pd.cut(vtr_total_df.index,bins)).mean()
vtr_time_bin_df.index = bin_centers
vtr_time_bin_df.head()

In [ ]:
test_df = vtr_time_bin_df.copy()

azi_avg_df = pd.DataFrame(np.zeros((test_df.shape[0],6)),
                         index = test_df.index,
                         columns = ['m' + str(m) for m in np.arange(6)+1])

weight = summed_azimuth_1+summed_azimuth_2
total_weight = np.sum(summed_azimuth_1 + summed_azimuth_2)

# weight = 1
# total_weight = 72

for m in np.arange(6):
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df['m'+str(m+1)] = test_df[stm_list].multiply(weight).sum(axis=1)/total_weight

In [ ]:
def plt_unix_time_to_CST(ax):
    plt.locator_params(axis='x', nbins=5)
    xticks = ax.get_xticks()
    ax.set_xticklabels([pd.to_datetime(tm, unit='s').tz_localize('UTC').tz_convert('US/Central').strftime('%Y-%m-%d\n %H:%M:%S %Z')
                          for tm in xticks], rotation=30)

m = 'm1'

fig, ax = plt.subplots(2,1)
plt.sca(ax[0])
plt.plot(azi_avg_df.index.values, azi_avg_df[m]/61.79, '.', color='navy', markersize=1)
plt.sca(ax[1])
plt.plot(azi_avg_df.index.values, azi_avg_df[m]/61.79, '.', color='navy', markersize=4)
ax[1].set_ylim(841.5, 841.8)
ax[0].set_ylim(840, 842)
ax[1].set_xlim(1524376800, 1524386400)
plt_unix_time_to_CST(ax[0])
plt_unix_time_to_CST(ax[1])

fig.set_size_inches(12,8)
fig.tight_layout()

In [ ]:
plt.plot(tr_baseline_1[:,0], '.')

In [ ]:
tr_run_3 = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3958_moment_df')

In [ ]:
print tr_run_3.index.values[-1]

In [ ]:
for i in weight: print np.round(i/2,1)